In [36]:
!pip install psycopg2-binary pgvector pypdf2 langchain openai

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 KB 1.7 MB/s eta 0:00:002.0 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 KB 5.6 MB/s eta 0:00:00


In [ ]:
'''SQL Commands for creating extention and table schema'''
# CREATE EXTENSION VECTOR;

# CREATE TABLE wordembed (
# 	id bigserial PRIMARY KEY,
# 	filename varchar,
# 	filetype varchar,
# 	filetext varchar,
# 	embedding vector(1536)
# );

# SELECT * FROM wordembed WHERE filename='doc_a' ORDER BY embedding <-> '[3,1,2]' LIMIT 5;

# DROP TABLE Items;


# TRUNCATE items;

# ALTER TABLE WORDEMBED ALTER COLUMN embedding TYPE vector(1536);

In [80]:
import openai
import numpy as np
import psycopg2 as pg
import psycopg2.extras as extras
import pandas as pd
from PyPDF2 import PdfReader

from langchain.text_splitter import CharacterTextSplitter

In [56]:
conn = pg.connect(host="localhost", dbname="embeddingdb", user="postgres", password="")

In [57]:
conn.autocommit = True

In [58]:
cursor = conn.cursor()

In [107]:
cursor.execute("SELECT * FROM wordembed")

In [108]:
cursor.fetchall()

[(1,
  'trulicity-uspi',
  'text',
  'HIGHLIGHTS OF PRESCRIBING INFORMATION  \nThese highlights do not include all the information needed to use \nTRULICITY  safely and effectively. See full prescribing information \nfor TRULICITY . \n \nTRULICITY  (dulaglutide ) injection, for  subcutaneous use  \nInitial U.S. Approval: 2014  \n \nWARNING: RISK OF THYROID C -CELL TUMORS  \nSee full prescribing information for complete boxed warning.  \n \n• Dulaglutide  causes thyroid C -cell tumors in rats . It is unknown \nwhether TRULICITY  causes thyroid C -cell tumors, including \nmedullary thyroid carcinoma (MTC), in humans  as the human \nrelevance of dulaglutide -induced rodent thyroid C -cell tumors  \nhas not be en determined  (5.1, 13.1). \n• TRULICITY  is contraindicated in patients with a personal or',
  '[-0.024307873,0.0013218017,0.010287863,-0.0075691277,0.005814879,0.0177242,-0.028347537,-0.031227019,-0.022616526,-0.011378153,0.02788626,0.026474474,-0.002755428,0.004867865,-0.00370069

In [13]:
text = ""
with open('trulicity-uspi.pdf', 'rb') as file:
    pdfreader = PdfReader(file)
    for page in pdfreader.pages:
        text+= page.extract_text()

In [18]:
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=800,
    chunk_overlap=200,
    length_function=len
)

In [19]:
chunks = text_splitter.split_text(text)

In [21]:
type(chunks)

list

In [33]:
df = pd.DataFrame(columns=['filename', 'filetype', 'filetext', 'embedding'])

In [38]:
openai.api_key=''

In [43]:
def gen_embedding(prompt):
    response = openai.Embedding.create(
        input=prompt,
        model="text-embedding-ada-002",
        
    )
    return response['data'][0]['embedding']

In [34]:
df['filetext'] = chunks
df['filename'] = 'trulicity-uspi'
df['filetype'] = 'text'

In [1]:
df['embedding'] = [gen_embedding(x) for x in df['filetext']]

In [104]:
type(df['embedding'][0])

list

In [45]:
df.to_csv('file_embeddings.csv')

In [106]:
# df is the dataframe
if len(df) > 0:
    df_columns = list(df)
    # create (col1,col2,...)
    columns = ",".join(df_columns)
    
    # create VALUES('%s', '%s",...) one '%s' per column
    values = "VALUES({})".format(",".join(["%s" for _ in df_columns])) 
    
    #create INSERT INTO table (columns) VALUES('%s',...)
    insert_stmt = "INSERT INTO {} ({}) {}".format('wordembed',columns,values)
    print(insert_stmt)
    extras.execute_batch(cursor, insert_stmt, df.values)

INSERT INTO wordembed (filename,filetype,filetext,embedding) VALUES(%s,%s,%s,%s)
